Praktikum NLP:

ToDo:
- Load dataset √

- Dependency Parsing √

- Extracting aspects (phrases/sentiment-words)

    - Summarize aspects for each hotel

    - Calculate sentiment scores for each aspect (TextBlob, VADER, etc...) √

- Analyse 5 most common aspects

- Dump into graphs for the presentation

In [69]:
#Imports and load data

import pandas as pd
import re
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
import string
from collections import Counter
from textblob import TextBlob

nlp = spacy.load("en_core_web_sm")

#nltk.download('wordnet')
#nltk.download('omw-1.4')

with open('../../include/Hotel_Reviews.csv') as file:
#with open('Hotel_Reviews.csv') as file:
    df = pd.read_csv(file, nrows=100000)

#!pip install spacy
#!python -m spacy download en_core_web_sm

In [70]:
## Clean data

# Delete redundant and empty reviews

def preprocessing_reviews(df):
    df.drop(df[ df['Negative_Review'] == 'No Negative'].index)
    df.drop(df[ df['Positive_Review'] == 'No Positive'].index)
    df.drop_duplicates(keep=False)

# Split into sentences and add dots to positive reviews and negative reviews
    for text in df['Positive_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t |n\'t ', ' not',complete_review)
        df['Positive_Review'] = df['Positive_Review'].replace([text], complete_review)

    for text in df['Negative_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t |n\'t ', ' not',complete_review)
        #        complete_review = re.sub(r'n\s+t|n\'t|nt ', ' not',complete_review)
        df['Negative_Review'] = df['Negative_Review'].replace([text], complete_review)
    return df

df = preprocessing_reviews(df)
pd.set_option("display.max_colwidth", None) # -1?

#TODO: Sentence splitting is sometimes inaccurate:
# My room was dirty and. I was afraid to walk barefoot o nothe floor 

# Quick check:
#print(df['Negative_Review'])
#print(df['Positive_Review'])

# Remove stopwords

#nlp = spacy.load("en_core_web_sm")
#stopword_list = nltk.corpus.stopwords.words('english')

print(df)

                                                   Hotel_Address  \
0       s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
1       s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
2       s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
3       s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
4       s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
...                                                          ...   
99995                20 Rue Jean Rey 15th arr 75015 Paris France   
99996                20 Rue Jean Rey 15th arr 75015 Paris France   
99997                20 Rue Jean Rey 15th arr 75015 Paris France   
99998                20 Rue Jean Rey 15th arr 75015 Paris France   
99999                20 Rue Jean Rey 15th arr 75015 Paris France   

       Additional_Number_of_Scoring Review_Date  Average_Score  \
0                               194    8/3/2017            7.7   
1                               194    8/3/2017    

In [71]:
# maybe check for words with "...nt" and then split the nt. Then check, if it's a verb. If yes, change from "didnt" to "did not" with a regex. (If not, leave it)

# since in the data there already is a distinction in positive and negative reviews we keep this and make two arrays
aspects_pos = []
aspects_neg = []

# extract all nouns, their adjectives, their adverbs for the negative reviews
for review in df["Negative_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False # TODO: re-test older version with parsed dependencies to find negations now that the data is cleaner...
    for negative_token in doc: # ... should make this faster, too
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
    
    # get all the nouns as aspects
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text

    # and the adjectives as descriptions...
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        # ... they can be modified by adverbs, so add them, too
        if child.pos_ != 'ADV': 
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text
  
  # pour into the array and make the dataframe
  aspects_neg.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_neg = pd.DataFrame(aspects_neg)
#print(pd.DataFrame(aspects_neg))


# extract all nouns, their adjectives, their adverbs for the positive reviews
for review in df["Positive_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False # TODO: re-test older version with parsed dependencies to find negations now that the data is cleaner
    for negative_token in doc: # ... should make this faster, too
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
        
    # get all the nouns as aspects
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text

    # and the adjectives as descriptions...
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        # ... they can be modified by adverbs, so add them, too
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text

  # pour into the array and make the dataframe
  aspects_pos.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_pos = pd.DataFrame(aspects_pos)
#print(pd.DataFrame(aspects_pos))

In [72]:
# just a quick check to see that the negations are currently not working that well
for boolean in aspects_neg['negation']:
    if boolean == True: print("true")

for boolean in aspects_pos['negation']:
    if boolean == True: print("true")


In [73]:
# we cut out the negation column here, since we will try to add that directly into the description later
cleanedDF = []
for aspect, descriptor, negation in aspects_pos.values:
    if aspect != '' and descriptor != '': cleanedDF.append({'aspect': aspect,
    'description': descriptor})

cleanedAspectsPos = pd.DataFrame(cleanedDF)

cleanedDF = []
for aspect, descriptor, negation in aspects_neg.values:
    if aspect != '' and descriptor != '': cleanedDF.append({'aspect': aspect,
    'description': descriptor})

cleanedAspectsNeg = pd.DataFrame(cleanedDF)
cleanedAspectsNeg


,aspect,description
0,room,newer
1,rooms,empty
2,offer,ready
3,photos,happy
4,mess,total
...,...,...
32579,room,small
32580,location,perfect
32581,room,too small
32582,Bathroom,other


In [74]:
# here we create the final aspects by grouping the ones we just extracted

# lemmatizing works better in this case
"""
stemmer = SnowballStemmer("english")
countStemmer = Counter()

for aspect in aspects['aspect']:
    aspect = stemmer.stem( aspect)
    aspect = aspect.lower()
    countStemmer[aspect] += 1

print(countStemmer)
"""


# let's count what we got so far
countLemmaPos = Counter()
countLemmaNeg = Counter()

# use the lemmatizer to remove unnessesary information
lemmatizer = WordNetLemmatizer()

for aspect in cleanedAspectsPos['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaPos[aspect] += 1


for aspect in cleanedAspectsNeg['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaNeg[aspect] += 1

print(countLemmaNeg, "\n")
print(countLemmaPos)

    

Counter({'room': 4522, 'staff': 1881, 'hotel': 1380, 'breakfast': 1162, 'bed': 864, 'bathroom': 766, 'rooms': 664, 'service': 499, 'price': 468, 'shower': 424, 'food': 381, 'location': 312, 'bar': 309, 'people': 299, 'area': 297, 'water': 276, 'wifi': 252, 'thing': 245, 'window': 235, 'view': 229, 'restaurant': 220, 'conditioning': 207, 'size': 200, 'lift': 192, 'door': 191, 'one': 182, 'parking': 170, 'facility': 161, 'receptionist': 152, 'pillow': 149, 'noise': 146, 'reception': 145, 'guest': 136, 'stay': 132, 'floor': 131, 'carpet': 130, 'toilet': 128, 'elevator': 126, 'bedroom': 125, 'furniture': 122, 'mattress': 118, 'light': 118, 'place': 114, 'pillows': 109, 'manager': 108, 'heating': 107, 'coffee': 105, 'member': 102, 'internet': 101, 'tea': 99, 'cost': 96, 'person': 96, 'system': 95, 'menu': 92, 'temperature': 91, 'housekeeping': 91, 'quality': 91, 'wall': 85, 'choice': 84, 'decor': 84, 'beds': 81, 'pool': 81, 'space': 80, 'egg': 79, 'alarm': 78, 'charge': 73, 'building': 72, 

In [75]:
sentiment_pos = []
for aspect in cleanedAspectsPos['description']:
  sentiment_pos.append(TextBlob(aspect).sentiment)
cleanedAspectsPos['sentiment'] = sentiment_pos

sentiment_neg = []
for aspect in cleanedAspectsNeg['description']:
  sentiment_neg.append(TextBlob(aspect).sentiment)
cleanedAspectsNeg['sentiment'] = sentiment_neg


In [79]:
print(cleanedAspectsPos)

         aspect       description    sentiment
0          park         beautiful  (0.85, 1.0)
1        option         fantastic   (0.4, 0.9)
2         range              nice   (0.6, 1.0)
3      building           outdoor   (0.0, 0.0)
4          room         beautiful  (0.85, 1.0)
...         ...               ...          ...
40041     Staff       informative   (0.0, 0.0)
40042     staff       very polite   (0.2, 0.3)
40043  Location              safe   (0.5, 0.5)
40044       bed  very comfortable  (0.52, 1.0)
40045   service           helpful   (0.0, 0.0)

[40046 rows x 3 columns]


In [77]:
'''
import pandas as pd
  
# initialize list of lists
data = [['room', 'too small', 0], ['wifi', 'not free', 0],['gym facilities', 'not free', 0],['food', 'terrible small', 0],['food', 'cold', 0],['drinks', 'cold', 0]]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Aspect', 'Description', 'Sentiment'])
df

sentiment = []
for aspect, des, sent in df.values:
  sentiment.append(TextBlob(des + aspect).sentiment)
df['Sentiment'] = sentiment

df
'''


"\nimport pandas as pd\n  \n# initialize list of lists\ndata = [['room', 'too small', 0], ['wifi', 'not free', 0],['gym facilities', 'not free', 0],['food', 'terrible small', 0],['food', 'cold', 0],['drinks', 'cold', 0]]\n  \n# Create the pandas DataFrame\ndf = pd.DataFrame(data, columns = ['Aspect', 'Description', 'Sentiment'])\ndf\n\nsentiment = []\nfor aspect, des, sent in df.values:\n  sentiment.append(TextBlob(des + aspect).sentiment)\ndf['Sentiment'] = sentiment\n\ndf\n"

In [78]:
# Define rules for extracting aspects: keywords:

# Aspect food: restaurant, breakfast, dinner, lunch, bar
# Aspect helpfulness: staff, personell, clerk, service, check-in
# Aspect price: cheap, expensive, euros, ...?
# Aspect cleanliness: bed, matress, bathroom, floor, sheets, "changed daily", shower
# Aspect features/equipment:  USB-port, wifi, airconditioning, showergel, shampoo, furniture, roomservice, bathtub
# Aspect connectivity: bus, tram, train, airportshuttle, parking
# Aspect property: stairs (steep), building, (under-)construction, renovation, location, citycenter, sightseeing


Presentation: Spatial-Mapping where hotels are // where guests are from
=> maybe what aspects are mentioned by primarily by guests by nationality or positive/negative in general

